In [1]:
import os
import h5py
import cv2
import numpy as np
import pandas as pd
import sklearn as skl
from matplotlib import pyplot as plt

from pdf2image import convert_from_path, convert_from_bytes
import pytesseract

from OCRVision import OCRVision
from OCRPharse3 import OCRPharse3
from OCRData import OCRData

In [2]:
# set home directory
home_path = os.getcwd()[:os.getcwd().find('Code/python/obf_autax/ocr')-1]
# set input_path
input_path = os.path.join(home_path, 'Data', 'obf_autman', 'pdf', 'Manuale_Johannklause_20201109.pdf')

# Manuale_Faistenau_20201101
# Manuale_Haselbach_Mannersdorf_20200430

In [3]:
# convert pdf to image (pdf2image)
def convert_to_image(input_path, page_nr):

    pages = convert_from_path(input_path, 500, first_page=page_nr, last_page=page_nr)

    return pages[0]

In [4]:
#img = convert_to_image(input_path, 3).convert('RGB')
#img = np.array(img)
# 113

img = convert_to_image(input_path, 1).convert('RGB')
img = np.array(img)

In [5]:
data = OCRPharse3()

In [6]:
data.pharse_page(img)

001 A 1


In [7]:
for i in range(2,1192): # 952
    img = convert_to_image(input_path, i).convert('RGB')
    img = np.array(img)
    data.pharse_page(img)

001 A2
001 A 3
001 B1
001 B2
001 B3
001 B4
001 B5
001 CO
001 D1
001 D2
001 D3
001 D4
001 D5
001 D6
001 D7’
001 EO
002 AO
002 BO
002 C 1
002 C 2
002 C 3
002 C4
002 C5

002 D 1
002 D2
002 E 1
002 E2
002 E3
002 E4
002 F 1
002 F2
003 A 1
003 A2

003 B 1
003 B2
003 B3
003 B4
003 CO
003 D 1
003 D2
003 D3
003 D4
003 E 0O
003 FO
003 G 1
003 G2
003 HO
010 A 1
010 A2
010 B1
010 B2
010 B3
010 C1
010 C2
011 A 1
011 A2
011 A 3
011 B1
011 B2
011 C1
011 C2
011 D1
011 D2
011 E1
011 E2
011 F1
011 F2
011 F3
011 G1
011 G2
011 G 3
012 A 1
012 A2
012 A3
012 A4
012 A5
012 A6
012 A 7
012 A383

012 A9
012 BO
012 C1
012 C2

012 DO
012 E1
012 E2
012 FO

013 A 1
013 A2
013 A3
013 A4
013 A5
013 A6
013 A 7
013 A8
013 Bi
013 B2
013 B3
013 B4
013 B5
013 B6
013 B7
013 CO
013 D1
013 D2
013 D3
013 E1
013 E2
013 E3
013 E4
013 E5
013 E6
014 A 1
014 A2
014 A 3
014 AA
014 A5
014 C1
014 C2
014 C 3
014 C4
014 DO
014 E 1
014 E2
014 F1
014 F2
014 F3
014 G 1
014 G2
014 G 3
015 A 1
015 A2
015 A3
015 B1
015 B2
015 B3
015 B4
015 B

In [8]:
data.bz

[['001A1', '4 FI', '2 LA', '3 BU', '1 TA', ''],
 ['001A2', '5 SFI', '2 LA', '1 TA', '2 BU', ''],
 ['001A3', '3 FI', '2 LA', '1 TA', '3 BU', '1 AH'],
 ['001B1', '4 FI2LA', '1 TA', '2 BU', '1 AH', ''],
 ['001B2', '5 FI', '2 LA', '2 BU', '1 AH', ''],
 ['001B3', '5 FI2LA', '2 BU', '1 AH', '', ''],
 ['001B4', '5 FI2LA', '1 TA', '2 BU', '', ''],
 ['001B5', '5 FI', '2 LA', '1 TA', '2 BU', ''],
 ['001C0', '5 FI2LA', '1 TA', '3 BU', '', ''],
 ['001D1', '4 FI', '2 LA', '1 TA', '3 BU', ''],
 ['001D2', '4 FI', '2 LA', '1 TA', '3 BU', ''],
 ['001D3', '4 FI', '3 BU', '3 LA', '', ''],
 ['001D4', '4 FI', '3 BU', '3 LA', '', ''],
 ['001D5', '', '', '', '', ''],
 ['001D6', '4 FI', '2 LA', '1 TA', '2 BU', ''],
 ['001D7’', '5 FI', '2 LA', '3 BU', '', ''],
 ['001E0', '', '', '', '', ''],
 ['002A0', '4 FI', '2 LA', '1 TA', '3 BU', ''],
 ['002B0', '4 FI', '2 LA', '1 TA', '3 BU', ''],
 ['002C1', '4 FI', '3 BU', '2 LA', '1 TA', ''],
 ['002C2', '4 FI', '3 BU', '2 LA', '1 TA', ''],
 ['002C3', '5 FI', '2 LA', '1 

In [9]:
data.wo_data

[['001A1', 0, 0, 0, 'SL', 0],
 ['001A2', 0, 0, 22, 'B', 0],
 ['001A3', 0, 0, 22, 'SL', 0],
 ['001B1', 0, 0, 21, '', 0],
 ['001B2', 0, 0, 0, '', 0],
 ['001B3', 0, 0, 0, '', 0],
 ['001B4', 0, 0, 0, '', '3'],
 ['001B5', 0, 0, 0, '', 0],
 ['001C0', 0, 0, 0, 'AL', 0],
 ['001D1', 0, 0, 0, '', 0],
 ['001D2', 0, 0, 0, '', 0],
 ['001D3', 0, 0, 0, '', 0],
 ['001D4', 0, 0, 0, '', 0],
 ['001D5', 0, 0, 0, '', 0],
 ['001D6', 0, 0, 22, '', 0],
 ['001D7’', 0, 0, 0, '', 0],
 ['001E0', 0, 0, 0, '', 0],
 ['002A0', 0, 200, 0, '', 0],
 ['002B0', 0, 0, 22, 'BH', 0],
 ['002C1', 0, 0, 0, '', 0],
 ['002C2', 0, 0, 0, '', 0],
 ['002C3', 0, 0, 0, '', 0],
 ['002C4', 0, 0, 0, '', 0],
 ['002C5', 0, 0, 23, '', 0],
 ['002D1', 0, 0, 22, 'B', 0],
 ['002D2', 0, 0, 0, 'SL', 0],
 ['002E1', 0, 0, 0, '', 0],
 ['002E2', 0, 0, 0, '', 0],
 ['002E3', 0, 0, 0, '', 0],
 ['002E4', 0, 0, 0, '', 0],
 ['002F1', 0, 0, 0, '', 0],
 ['002F2', 0, 0, 0, '', 0],
 ['003A1', 0, 0, 0, '', 0],
 ['003A2', 0, 0, 0, '', 0],
 ['003B1', 0, 0, 21, 'SL

In [9]:
# get SAP raw data

In [11]:
# set raw_path
raw_path = os.path.join(home_path, 'Data', 'obf_autman', 'raw_data', 'TO_1366_181_08_20201111.XLS')
# read tsv
wo_data = pd.read_csv(raw_path, sep='\t', encoding = "ISO-8859-1", decimal=',', error_bad_lines=False)
# filter relevant data
wo_data = wo_data[wo_data['Best.-Schicht'] == 0]

### pernitz
#wo_data = wo_data[wo_data['Abteilung'] > 93]

# sort data
wo_data.sort_values(by=['Abteilung', 'WE-Typ', 'Unterabteil.', 'Teilfl.'], ascending=[True,False,True,True], inplace=True)
wo_data = wo_data[['Waldort', 'WE-Typ']]

/home/philipp/miniconda3/envs/autax_rec/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (149) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### write to disk

In [12]:
# loop over all data
for data_part in [[data.wo_data, 'wo_data', ['Waldort', 'bkl', 'uz', 'stoe', 'vtyp', 'vg']], \
                  [data.bz, 'bz', ['Waldort', 'BZ1', 'BZ2', 'BZ3', 'BZ4', 'BZ5']], \
                  [data.text, 'text', ['Waldort', 'text1', 'text2', 'text3', 'text4', 'text5']], \
                  [data.ma, 'ma', ['Waldort', 'S1', 'MA1', 'fl1', 'DR1', 'BH1', 'ZP1', 'SG1', 'RU1', 'Text1',\
                                   'S2', 'MA2', 'fl2', 'DR2', 'BH2', 'ZP2', 'SG2', 'RU2', 'Text2',\
                                   'S3', 'MA3', 'fl3', 'DR3', 'BH3', 'ZP3', 'SG3', 'RU3', 'Text3']]]:
    
    print('creating ' + data_part[1])
    
    # create a dataframe out of the list
    df = pd.DataFrame(data_part[0], columns=data_part[2])

    # merge
    merge_data = pd.merge(wo_data, df, how='left', on='Waldort')

    # specify the path for writing dow the data
    save_path = os.path.join(home_path, 'Data', 'obf_autman', 'csv', 'johannklause', data_part[1] + '_1366_08.csv')
    # write data to disk
    df.to_csv(save_path, index=False)

creating wo_data
creating bz
creating text
creating ma


In [31]:

for typ in ['bz','ma','text','wo_data']:
    print('merging ' + typ)
    # set raw_path
    path = os.path.join(home_path, 'Data', 'obf_autman', 'csv', 'johannklause', typ + '_1366_08.csv')
    # read tsv
    typ_data = pd.read_csv(path)
    # merge
    merge_data = pd.merge(wo_data, typ_data, how='left', on='Waldort')
    # specify the path for writing dow the data
    save_path = os.path.join(home_path, 'Data', 'obf_autman', 'csv', 'johannklause', 'x' + typ + '_1366_08.csv')
    # write data to disk
    merge_data.to_csv(save_path, index=False)

merging bz
merging ma
merging text
merging wo_data


In [32]:
# set raw_path
raw_path = os.path.join(home_path, 'Data', 'obf_autman', 'raw_data', 'TO_1364_178_02_20201111.XLS')
# read tsv
wo_data = pd.read_csv(raw_path, sep='\t', encoding = "ISO-8859-1", decimal=',', error_bad_lines=False)
# filter relevant data
wo_data = wo_data[wo_data['Best.-Schicht'] == 0]

### pernitz
wo_data = wo_data[wo_data['Abteilung'] > 93]

# sort data
wo_data.sort_values(by=['Abteilung', 'WE-Typ', 'Unterabteil.', 'Teilfl.'], ascending=[True,False,True,True], inplace=True)
wo_data = wo_data[['Waldort', 'WE-Typ']]

for typ in ['bz','ma','text','wo_data']:
    print('merging ' + typ)
    # set raw_path
    path = os.path.join(home_path, 'Data', 'obf_autman', 'csv', 'feistenau', typ + '_1364_02.csv')
    # read tsv
    typ_data = pd.read_csv(path)
    # merge
    merge_data = pd.merge(wo_data, typ_data, how='left', on='Waldort')
    # specify the path for writing dow the data
    save_path = os.path.join(home_path, 'Data', 'obf_autman', 'csv', 'feistenau', 'x' + typ + '_1364_02.csv')
    # write data to disk
    merge_data.to_csv(save_path, index=False)

merging bz
merging ma
merging text
merging wo_data


/home/philipp/miniconda3/envs/autax_rec/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (36,102,103,108,112,113,114,124,126,147) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
